# Conditions and to do's:
---
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
    - San Francisco: Financial District, South Of Market or Mission Bay
- Everyone in the company is between 25 and 40, give them some place to go party.
    - San Francisco: South Of Market -> 13 possible offices
- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
    - `[-122.394523, 37.764726]` in Potrero Hill  
---
To do:
- 30% of the company staff have at least 1 child.
    - To do: look for schools in/near South Of Market
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
    - To do: look for Starbucks in/near South Of Market
- Account managers need to travel a lot.
    - To do: look for renting car places in/near South Of Market
- The CEO is vegan.
    - To do: look for vegan places in/near South of Market
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
    - To do: look for a basketball stadium and check the distance to the possible offices
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.
    - To do: look for a pet hairdresser

# Libraries

In [1]:
import os
import requests
import json
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from dotenv import load_dotenv
import time
import math

In [12]:
def getCoordinates(address):
    url_geocode = f"https://geocode.xyz/{address}?json=1"
    res = requests.get(url_geocode)
    try:
        return res.json()
    except:
        time.sleep(4)
        getCoordinates(address)

In [13]:
res = getCoordinates("San Francisco")
res

{'standard': {'addresst': {},
  'statename': {},
  'city': 'San Francisco',
  'prov': 'US',
  'countryname': 'United States of America',
  'postal': {},
  'confidence': '0.90'},
 'longt': '-122.40507',
 'alt': {'loc': {'longt': '-122.426263377796',
   'prov': 'CA',
   'city': 'SAN FRANCISCO',
   'postal': '94112',
   'score': '24905',
   'latt': '37.6874300564143'}},
 'elevation': {},
 'latt': '37.75672'}

In [14]:
lat, lon = res["alt"]["loc"]["latt"], res["alt"]["loc"]["longt"]

In [15]:
lat, lon

('37.6874300564143', '-122.426263377796')

In [16]:
map_SF = Map(location = [lat, lon], zoom_start = 12)

In [17]:
map_SF

In [18]:
# These coordinates are not very accurate for the neighborhood of interest. I'm using the one found in wikipedia:
lat = 37.7772222
lon = -122.4111111
map_SF = Map(location = [lat, lon], zoom_start = 15)
map_SF

In [19]:
load_dotenv()

True

In [20]:
token = os.getenv("token")

In [21]:
#Size of South of Market: it is aproximately 2.3 miles long -> 3.7 km long

In [22]:
def request_4sq(venue, lat, lon, radius = 3700, sort_by = "DISTANCE", limit = 20):
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={lat}%2C{lon}&radius={radius}&sort={sort_by}&limit={limit}"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    try:
        return requests.get(url, headers = headers).json()
    except:
        print("Request not found")

In [23]:
school_request = request_4sq("school", lat, lon)

In [24]:
starbucks_request = request_4sq("starbucks", lat, lon)

In [25]:
car_request =request_4sq("car rental", lat, lon)

In [26]:
vegan_request =request_4sq("vegan", lat, lon)

In [27]:
basketball_request =request_4sq("basketball court", lat, lon)

In [28]:
dog_request =request_4sq("pet groomer", lat, lon)

# Schools

In [29]:
school_request["results"][3]["categories"][0]["name"]

'Daycare'

In [30]:
school_categ = [i["categories"][0]["name"] for i in school_request["results"]]
print(school_categ)

['College and University', 'Preschool', 'Technology Business', 'Daycare', 'Education', 'Language School', 'Elementary School', 'Preschool', 'College Academic Building', 'Music School', 'College and University', 'Daycare', 'Law School', 'Education', 'College Academic Building', 'Preschool', 'Non-Profit Organization', 'Education', 'Education', 'Public Art']


In [31]:
school_lat = [i["geocodes"]["main"]["latitude"] for i in school_request["results"]]

In [32]:
school_lon = [i["geocodes"]["main"]["longitude"] for i in school_request["results"]]

In [33]:
school_open = [i["closed_bucket"] for i in school_request["results"]]
print(school_open)

['LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyClosed', 'VeryLikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'Unsure', 'LikelyOpen', 'Unsure', 'LikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'Unsure', 'LikelyClosed', 'Unsure', 'LikelyOpen', 'LikelyOpen', 'Unsure', 'LikelyOpen']


In [34]:
df_schools = pd.DataFrame({"school_categ": school_categ, "school_lat": school_lat, "school_lon": school_lon, "school_open": school_open})

In [35]:
df_schools

,school_categ,school_lat,school_lon,school_open
0,College and University,37.777085,-122.410024,LikelyOpen
1,Preschool,37.779426,-122.411882,VeryLikelyOpen
2,Technology Business,37.779378,-122.410374,VeryLikelyClosed
3,Daycare,37.780990,-122.419214,VeryLikelyOpen
4,Education,37.774363,-122.413160,LikelyOpen
5,Language School,37.776338,-122.406969,VeryLikelyOpen
6,Elementary School,37.776344,-122.406468,VeryLikelyOpen
7,Preschool,37.773203,-122.413809,Unsure
8,College Academic Building,37.774589,-122.416184,LikelyOpen
9,Music School,37.781455,-122.407558,Unsure


In [36]:
indexes = df_schools["school_categ"][df_schools["school_categ"].isin(["College and University", "Technology Business", "Language School", 
                                               "College Academic Building", "Music School", "College and University", "Law School",
                                               "College Academic Building", "Public Art"])].index
df_schools.drop(indexes, inplace=True)
indexes = df_schools["school_open"][df_schools["school_open"].isin(["Unsure", "LikelyClosed"])].index
df_schools.drop(indexes, inplace=True)
df_schools.drop(columns = "school_open", axis = 1, inplace = True )
df_schools

,school_categ,school_lat,school_lon
1,Preschool,37.779426,-122.411882
3,Daycare,37.780990,-122.419214
4,Education,37.774363,-122.413160
6,Elementary School,37.776344,-122.406468
11,Daycare,37.773386,-122.414823
16,Non-Profit Organization,37.783915,-122.411287
17,Education,37.776696,-122.419821


# Starbucks

In [37]:
starbucks_request["results"]

[{'fsq_id': '49bee717f964a520f3541fe3',
  'categories': [{'id': 13035,
    'name': 'Coffee Shop',
    'short_name': 'Coffee Shop',
    'plural_name': 'Coffee Shops',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'}}],
  'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
    'name': 'Starbucks'}],
  'closed_bucket': 'VeryLikelyOpen',
  'distance': 543,
  'geocodes': {'main': {'latitude': 37.777088, 'longitude': -122.417264},
   'roof': {'latitude': 37.777088, 'longitude': -122.417264}},
  'link': '/v3/places/49bee717f964a520f3541fe3',
  'location': {'address': '1390 Market St',
   'address_extended': 'Ste 107',
   'census_block': '060750124053010',
   'country': 'US',
   'cross_street': 'at Fox Plz',
   'dma': 'San Francisco-Oakland-San Jose',
   'formatted_address': '1390 Market St (at Fox Plz), San Francisco, CA 94102',
   'locality': 'San Francisco',
   'postcode': '94102',
   'region': 'CA'},
  'name': 'Starbucks',
  're

In [38]:
starbucks_lat = [i["geocodes"]["main"]["latitude"] for i in starbucks_request["results"]]
starbucks_lon = [i["geocodes"]["main"]["longitude"] for i in starbucks_request["results"]]
starbucks_open = [i["closed_bucket"] for i in starbucks_request["results"]]
print(starbucks_open)

['VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen']


In [39]:
df_starbucks = pd.DataFrame({"starbucks_lat": starbucks_lat, "starbucks_lon": starbucks_lon})

# Car rentals

In [40]:
car_request["results"][8]["name"]

'Avis Rent A Car'

In [41]:
car_name = [i["name"] for i in car_request["results"]]
car_lat = [i["geocodes"]["main"]["latitude"] for i in car_request["results"]]
car_lon = [i["geocodes"]["main"]["longitude"] for i in car_request["results"]]
car_open = [i["closed_bucket"] for i in car_request["results"]]
print(car_open)

['Unsure', 'Unsure', 'LikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'Unsure', 'LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'LikelyOpen', 'LikelyOpen', 'Unsure', 'Unsure', 'LikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'Unsure', 'Unsure', 'Unsure']


In [42]:
df_cars = pd.DataFrame({"car_name": car_name, "car_lat": car_lat, "car_lon": car_lon, "car_open": car_open})
df_cars

,car_name,car_lat,car_lon,car_open
0,Zipcar Strata at Mission Bay,37.777886,-122.414085,Unsure
1,Zipcar,37.777848,-122.414155,Unsure
2,Alamo,37.774613,-122.409878,LikelyOpen
3,Lyft Rentals,37.774521,-122.409801,LikelyOpen
4,Budget Rent A Car,37.785633,-122.410600,VeryLikelyOpen
5,Enterprise Rent-A-Car,37.772257,-122.419344,VeryLikelyOpen
6,Zipcar Cova Hotel,37.784129,-122.417034,Unsure
7,Budget Rent A Car,37.782649,-122.403040,LikelyOpen
8,Avis Rent A Car,37.782742,-122.403117,VeryLikelyOpen
9,Avis Rent A Car,37.785633,-122.410600,VeryLikelyOpen


In [43]:
indexes = df_cars["car_open"][df_cars["car_open"] =="Unsure"].index
df_cars.drop(indexes, inplace=True)
df_cars.drop(columns="car_open", inplace=True)

# Vegan restaurants

In [44]:
vegan_request["results"] # -> Not all of them are exclusively vegan, but they offer vegan food

[{'fsq_id': '5c09e17b894eac002c3810b9',
  'categories': [{'id': 13064,
    'name': 'Pizzeria',
    'short_name': 'Pizza',
    'plural_name': 'Pizzerias',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
     'suffix': '.png'}},
   {'id': 13388,
    'name': 'Wings Joint',
    'short_name': 'Wings',
    'plural_name': 'Wings Joints',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/wings_',
     'suffix': '.png'}}],
  'chains': [],
  'closed_bucket': 'LikelyOpen',
  'distance': 224,
  'geocodes': {'main': {'latitude': 37.77953, 'longitude': -122.410096},
   'roof': {'latitude': 37.77953, 'longitude': -122.410096}},
  'link': '/v3/places/5c09e17b894eac002c3810b9',
  'location': {'address': '1077 Mission St',
   'census_block': '060750176021000',
   'country': 'US',
   'cross_street': '7th Street',
   'dma': 'San Francisco-Oakland-San Jose',
   'formatted_address': '1077 Mission St (7th Street), San Francisco, CA 94103',
   'locality': 'San Fr

In [45]:
vegan_name = [i["name"] for i in vegan_request["results"]]
vegan_lat = [i["geocodes"]["main"]["latitude"] for i in vegan_request["results"]]
vegan_lon = [i["geocodes"]["main"]["longitude"] for i in vegan_request["results"]]
vegan_open = [i["closed_bucket"] for i in vegan_request["results"]]
print(vegan_open)

['LikelyOpen', 'LikelyOpen', 'Unsure', 'Unsure', 'Unsure', 'LikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'LikelyOpen', 'Unsure', 'Unsure', 'LikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'LikelyOpen', 'LikelyOpen', 'VeryLikelyOpen']


In [46]:
df_vegan = pd.DataFrame({"vegan_name": vegan_name, "vegan_lat": vegan_lat, "vegan_lon": vegan_lon, "vegan_open": vegan_open})
df_vegan

,vegan_name,vegan_lat,vegan_lon,vegan_open
0,Square Pie Guys,37.779530,-122.410096,LikelyOpen
1,Sightglass Coffee,37.776972,-122.408512,LikelyOpen
2,Ted’s Market,37.772860,-122.416154,Unsure
3,Golden Era Vegan Restaurant,37.781213,-122.416719,Unsure
4,Costco,37.770612,-122.410932,Unsure
5,Mr.East Kitchen,37.780427,-122.403913,LikelyOpen
6,Souley Vegan,37.776546,-122.402537,LikelyOpen
7,Sweetgreen,37.782400,-122.405048,VeryLikelyOpen
8,SOMA StrEat Food Park,37.769745,-122.412028,LikelyOpen
9,Rad Radish,37.776891,-122.421553,VeryLikelyOpen


In [47]:
indexes = df_vegan["vegan_open"][df_vegan["vegan_open"] =="Unsure"].index
df_vegan.drop(indexes, inplace=True)
df_vegan.drop(columns = "vegan_open", inplace=True)

# Basketball

In [48]:
basketball_request["results"][0] # -> not stadiums, but courts

{'fsq_id': '4bc8a0ce15a7ef3b117f7bda',
 'categories': [{'id': 18008,
   'name': 'Basketball Court',
   'short_name': 'Basketball Court',
   'plural_name': 'Basketball Courts',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/basketballcourt_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'LikelyOpen',
 'distance': 462,
 'geocodes': {'drop_off': {'latitude': 37.77898, 'longitude': -122.406215},
  'main': {'latitude': 37.778838, 'longitude': -122.406328},
  'roof': {'latitude': 37.778838, 'longitude': -122.406328}},
 'link': '/v3/places/4bc8a0ce15a7ef3b117f7bda',
 'location': {'address': '6TH St',
  'census_block': '060750178041000',
  'country': 'US',
  'cross_street': 'Folsom Street',
  'dma': 'San Francisco-Oakland-San Jose',
  'formatted_address': '6TH St (Folsom Street), San Francisco, CA 94109',
  'locality': 'San Francisco',
  'postcode': '94109',
  'region': 'CA'},
 'name': 'Soma Rec Center',
 'related_places': {},
 'timezone': 'America/Lo

In [49]:
court_name = [i["name"] for i in basketball_request["results"]]
court_lat = [i["geocodes"]["main"]["latitude"] for i in basketball_request["results"]]
court_lon = [i["geocodes"]["main"]["longitude"] for i in basketball_request["results"]]
court_open = [i["closed_bucket"] for i in basketball_request["results"]]
print(court_open)

['LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'Unsure', 'VeryLikelyOpen', 'Unsure', 'LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'Unsure', 'VeryLikelyOpen', 'LikelyOpen']


In [50]:
df_court = pd.DataFrame({"court_name": court_name, "court_lat": court_lat, "court_lon": court_lon, "court_open": court_open})
df_court

,court_name,court_lat,court_lon,court_open
0,Soma Rec Center,37.778838,-122.406328,LikelyOpen
1,Basketball Court,37.781084,-122.414131,VeryLikelyOpen
2,Equinox Sports Club San Francisco,37.786373,-122.404335,VeryLikelyOpen
3,Berry Basketball Courts,37.772080,-122.397686,VeryLikelyOpen
4,Mission Creek Park,37.770963,-122.398624,VeryLikelyOpen
5,NBA Nation,37.759168,-122.412335,Unsure
6,Jose Coronado Playground,37.757714,-122.415116,VeryLikelyOpen
7,Hamilton Basketball Courts,37.785284,-122.435668,Unsure
8,Dolores Park Basketball Courts,37.761189,-122.427791,LikelyOpen
9,Chase Center Stadium,37.767924,-122.387478,VeryLikelyOpen


In [51]:
indexes = df_court["court_open"][df_court["court_open"] =="Unsure"].index
df_court.drop(indexes, inplace=True)
df_court.drop(columns= "court_open", inplace=True)

# Dogs

In [52]:
dog_request["results"][3]

{'fsq_id': '50d518e33b1d42680df236a9',
 'categories': [{'id': 17000,
   'name': 'Retail',
   'short_name': 'Retail',
   'plural_name': 'Retail',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'LikelyClosed',
 'distance': 488,
 'geocodes': {'drop_off': {'latitude': 37.775025, 'longitude': -122.415603},
  'main': {'latitude': 37.774817, 'longitude': -122.415768},
  'roof': {'latitude': 37.774817, 'longitude': -122.415768}},
 'link': '/v3/places/50d518e33b1d42680df236a9',
 'location': {'address': '130 10th St',
  'census_block': '060750176033005',
  'country': 'US',
  'dma': 'San Francisco-Oakland-San Jose',
  'formatted_address': '130 10th St, San Francisco, CA 94103',
  'locality': 'San Francisco',
  'postcode': '94103',
  'region': 'CA'},
 'name': 'Pet Club',
 'related_places': {}}

In [53]:
dogs_name = [i["name"] for i in dog_request["results"]]
dogs_lat = [i["geocodes"]["main"]["latitude"] for i in dog_request["results"]]
dogs_lon = [i["geocodes"]["main"]["longitude"] for i in dog_request["results"]]
dogs_open = [i["closed_bucket"] for i in dog_request["results"]]
print(dogs_open)

['VeryLikelyClosed', 'VeryLikelyClosed', 'VeryLikelyOpen', 'LikelyClosed', 'LikelyOpen', 'LikelyClosed', 'VeryLikelyClosed', 'VeryLikelyClosed', 'LikelyClosed', 'VeryLikelyOpen', 'Unsure', 'VeryLikelyClosed', 'VeryLikelyOpen', 'LikelyClosed', 'VeryLikelyClosed', 'Unsure', 'VeryLikelyOpen', 'LikelyClosed', 'VeryLikelyClosed', 'LikelyClosed']


In [54]:
df_dogs = pd.DataFrame({"dogs_name": dogs_name, "dogs_lat": dogs_lat, "dogs_lon": dogs_lon, "dogs_open": dogs_open})
df_dogs

,dogs_name,dogs_lat,dogs_lon,dogs_open
0,By Leaps & Bounds Pet &H,37.776857,-122.410440,VeryLikelyClosed
1,San Francisco Pet Sitters,37.776108,-122.413407,VeryLikelyClosed
2,Fuzzy Pet Health,37.775175,-122.415359,VeryLikelyOpen
3,Pet Club,37.774817,-122.415768,LikelyClosed
4,Pet Food Patrol,37.775455,-122.420249,LikelyOpen
5,Aussie Pet Mobile,37.781390,-122.420815,LikelyClosed
6,Hayes Valley Pet Care,37.773985,-122.421954,VeryLikelyClosed
7,Pet Pal Distribution Corporation,37.769265,-122.404187,VeryLikelyClosed
8,On The Go Pet Care,37.775123,-122.423312,LikelyClosed
9,Happy Pet Care Services,37.767062,-122.410794,VeryLikelyOpen


In [55]:
indexes = df_dogs["dogs_open"][df_dogs["dogs_open"].isin(["VeryLikelyClosed", "LikelyClosed", "Unsure"])].index
df_dogs.drop(indexes, inplace=True)
df_dogs.drop(columns = "dogs_open", inplace=True)

# Weighing priorities

According to the number of people affected by their expressed needs, they are prioritised:
- Everyone in the company is between 25 and 40, give them some place to go party. **DONE**
- 30% of the company staff (29 people) have at least 1 child. **school_df, PRIORITY 1**
- 20 Designers : Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design. **designSF_coordinates, PRIORITY 2**
- 20 Account Managers: Account managers need to travel a lot. **cars_df, PRIORITY 3**
- 15 Data Engineers
- 10 Frontend Developers, 5 Backend Developers: Developers like to be near successful tech startups that have raised at least 1 Million dollars. **DONE**
- 10 Executives: Executives like Starbucks A LOT. Ensure there's a starbucks not too far. **sctarbucks_df, PRIORITY 4**
- 5 UI/UX Engineers
- 1 Maintenance guy that loves basketball. **courts_df, PRIORITY 5**
- 1 CEO/President: The CEO is vegan. **vegan_df, PRIORITY 6**
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away. **dogs_df, PRIORITY 7**

# Distances

In [56]:
def distances(coordinates_1, coordinates_2):
    R = 6371000 # radius of Earth in meters
    lon_1, lat_1 = coordinates_1
    lon_2, lat_2 = coordinates_2
    phi_1 = math.radians(lat_1)
    phi_2 = math.radians(lat_2)
    delta_phi = math.radians(lat_2 - lat_1)
    delta_lambda = math.radians(lon_2 - lon_1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    km = round(R * c / 1000.0, 3)
    return km

In [57]:
list_offices = [{'type': 'Point', 'coordinates': [-122.391276, 37.7815]},
                {'type': 'Point', 'coordinates': [-122.3921923, 37.781381]},
                {'type': 'Point', 'coordinates': [-122.4097127, 37.78205]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.408109, 37.783521]},
                {'type': 'Point', 'coordinates': [-122.3920816, 37.7828482]},
                {'type': 'Point', 'coordinates': [-122.3920816, 37.7828482]},
                {'type': 'Point', 'coordinates': [-122.39586, 37.785766]}]

## Priority 1: Schools

In [59]:
school_coordinates = [[row["school_lat"], row["school_lon"]] for index, row in df_schools.iterrows()]
school_coordinates

[[37.779426, -122.411882],
 [37.78099, -122.419214],
 [37.774363, -122.41316],
 [37.776344, -122.406468],
 [37.773386, -122.414823],
 [37.783915, -122.411287],
 [37.776696, -122.419821]]

In [60]:
school_distances = [{"office":j, "distance": distances(i, [j["coordinates"][1],j["coordinates"][0]])} for j in list_offices for i in school_coordinates]
df_school_distance = pd.DataFrame(school_distances).sort_values(by = "distance", ascending = True)
df_school_distance

,office,distance
19,"{'type': 'Point', 'coordinates': [-122.4097127...",0.207
14,"{'type': 'Point', 'coordinates': [-122.4097127...",0.287
68,"{'type': 'Point', 'coordinates': [-122.408109,...",0.354
66,"{'type': 'Point', 'coordinates': [-122.408109,...",0.465
63,"{'type': 'Point', 'coordinates': [-122.408109,...",0.485
...,...,...
13,"{'type': 'Point', 'coordinates': [-122.3921923...",3.085
83,"{'type': 'Point', 'coordinates': [-122.3920816...",3.106
76,"{'type': 'Point', 'coordinates': [-122.3920816...",3.106
1,"{'type': 'Point', 'coordinates': [-122.391276,...",3.107


In [61]:
df_school_distance[df_school_distance["distance"] <= 0.5]["office"].value_counts()

office
{'type': 'Point', 'coordinates': [-122.4097127, 37.78205]}    3
{'type': 'Point', 'coordinates': [-122.408109, 37.783521]}    3
Name: count, dtype: int64

In [62]:
x = pd.DataFrame(df_school_distance[df_school_distance["distance"] <= 0.5]["office"].value_counts())
x.reset_index(inplace=True)

In [63]:
list_offices_updated = [row["office"] for index, row in x.iterrows()]
list_offices_updated

[{'type': 'Point', 'coordinates': [-122.4097127, 37.78205]},
 {'type': 'Point', 'coordinates': [-122.408109, 37.783521]}]

## Priority 2: design company

In [64]:
designSF_coordinates = [{'offices': [{'latitude': 37.764726, 'longitude': -122.394523}]}]
designSF_latlon = [designSF_coordinates[0]["offices"][0]["latitude"], designSF_coordinates[0]["offices"][0]["longitude"]]

In [65]:
design_distance = [{"office":i, "distance": distances(designSF_latlon, [i["coordinates"][1],i["coordinates"][0]])} for i in list_offices_updated]
design_distance
df_design_distance = pd.DataFrame(design_distance).sort_values(by = "distance", ascending = True)
df_design_distance["office"]

1    {'type': 'Point', 'coordinates': [-122.408109,...
0    {'type': 'Point', 'coordinates': [-122.4097127...
Name: office, dtype: object

In [66]:
df_design_distance = df_design_distance.head(1)

# Final office:

In [69]:
for index, row in df_design_distance.iterrows():
    print(row["office"])

{'type': 'Point', 'coordinates': [-122.408109, 37.783521]}


In [73]:
coordinates = [row["office"] for index, rows in df_design_distance.iterrows()][0]["coordinates"]
coordinates

[-122.408109, 37.783521]

In [77]:
rents = pd.read_csv("../data/officesSF.csv", encoding = "unicode_escape")
rents[rents["positions"]== str(coordinates)]

,employees,lat,lon,address,positions
143,112,37.783521,-122.408109,901 Market Street,"[-122.408109, 37.783521]"
